# Required Installations of Spark & Jave and set up of Environment Variables.

In [1]:
import os
# Find the latest version of spark 3.2 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()



Hit:1 http://archive.ubuntu.com/ubuntu focal InRelease
Get:2 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Get:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease [18.1 kB]
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:11 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [3,065 kB]
Hit:12 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:13 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1,320 kB

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.5.1.jar

--2023-03-27 19:11:48--  https://jdbc.postgresql.org/download/postgresql-42.5.1.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1046770 (1022K) [application/java-archive]
Saving to: ‘postgresql-42.5.1.jar’

postgresql-42.5.1.j 100%[===================>]   1022K  6.28MB/s    in 0.2s    

2023-03-27 19:11:48 (6.28 MB/s) - ‘postgresql-42.5.1.jar’ saved [1046770/1046770]



# Load Amazon Data into Spark DataFrame
## Get the Country latitude and longitude details with GDP

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("FinalProject-WomenWellBeing").config("spark.driver.extraClassPath","/content/postgresql-42.5.1.jar").getOrCreate()


In [4]:
from pyspark import SparkFiles
url = "https://mywomenwellbeing.s3.us-west-1.amazonaws.com/world_country_and_usa_states_latitude_and_longitude_values.csv"
spark.sparkContext.addFile(url)



from pyspark.sql.types import StructType,StructField,StringType,IntegerType,DoubleType
schema = StructType([
  StructField("country_code", StringType(), False),  
  StructField("latitude", DoubleType(), True),
  StructField("longitude", DoubleType(), True),
  StructField("country_name", StringType(), False),
  ])

# country_df = spark.read.csv(SparkFiles.get("world_country_and_usa_states_latitude_and_longitude_values.csv"), sep=",", header=True)
country_df = spark.read.option("encoding", "UTF-8").schema(schema).csv(SparkFiles.get("world_country_and_usa_states_latitude_and_longitude_values.csv"), sep=",", header=True, inferSchema=True)
country_df.show()


+------------+----------+-----------+--------------------+
|country_code|  latitude|  longitude|        country_name|
+------------+----------+-----------+--------------------+
|          AD| 42.546245|   1.601554|             Andorra|
|          AE| 23.424076|  53.847818|United Arab Emirates|
|          AF|  33.93911|  67.709953|         Afghanistan|
|          AG| 17.060816| -61.796428| Antigua and Barbuda|
|          AI| 18.220554| -63.068615|            Anguilla|
|          AL| 41.153332|  20.168331|             Albania|
|          AM| 40.069099|  45.038189|             Armenia|
|          AN| 12.226079| -69.060087|Netherlands Antilles|
|          AO|-11.202692|  17.873887|              Angola|
|          AQ|-75.250973|  -0.071389|          Antarctica|
|          AR|-38.416097| -63.616672|           Argentina|
|          AS|-14.270972|-170.132217|      American Samoa|
|          AT| 47.516231|  14.550072|             Austria|
|          AU|-25.274398| 133.775136|           Australi

### Load the dataframe with required columns and clean the data.

In [5]:
country_clean_df  = country_df.select(country_df['country_code'],country_df['country_name'],country_df['latitude'],country_df['longitude'])
country_clean_df.describe().show()
country_clean_df.dropna().describe().show()

CountryDemographics = country_clean_df.dropna()

+-------+------------+------------+------------------+------------------+
|summary|country_code|country_name|          latitude|         longitude|
+-------+------------+------------+------------------+------------------+
|  count|         245|         245|               244|               244|
|   mean|        null|        null|  16.2531093647541|13.294813586065576|
| stddev|        null|        null|27.031206079229964| 73.97647657765005|
|    min|          AD| Afghanistan|        -75.250973|       -177.156097|
|    max|          ZW|    Zimbabwe|         77.553604|        179.414413|
+-------+------------+------------+------------------+------------------+

+-------+------------+------------+------------------+------------------+
|summary|country_code|country_name|          latitude|         longitude|
+-------+------------+------------+------------------+------------------+
|  count|         244|         244|               244|               244|
|   mean|        null|        null|  

In [6]:
CountryDemographics.show()

+------------+--------------------+----------+-----------+
|country_code|        country_name|  latitude|  longitude|
+------------+--------------------+----------+-----------+
|          AD|             Andorra| 42.546245|   1.601554|
|          AE|United Arab Emirates| 23.424076|  53.847818|
|          AF|         Afghanistan|  33.93911|  67.709953|
|          AG| Antigua and Barbuda| 17.060816| -61.796428|
|          AI|            Anguilla| 18.220554| -63.068615|
|          AL|             Albania| 41.153332|  20.168331|
|          AM|             Armenia| 40.069099|  45.038189|
|          AN|Netherlands Antilles| 12.226079| -69.060087|
|          AO|              Angola|-11.202692|  17.873887|
|          AQ|          Antarctica|-75.250973|  -0.071389|
|          AR|           Argentina|-38.416097| -63.616672|
|          AS|      American Samoa|-14.270972|-170.132217|
|          AT|             Austria| 47.516231|  14.550072|
|          AU|           Australia|-25.274398| 133.77513

### Load the Country GDP csv.
### Drop the unnecessary columns and drop the null values and store the clean data into a new dataframe.

In [7]:

from pyspark import SparkFiles
url = "https://mywomenwellbeing.s3.us-west-1.amazonaws.com/Country_GDP.csv"

spark.sparkContext.addFile(url)


from pyspark.sql.types import StructType,StructField,StringType,IntegerType,DoubleType
countryGDPSchema = StructType([
  StructField("country_code", StringType(), False),
  StructField("country_name", StringType(), False),
  
  StructField("2000", IntegerType(), True),
  StructField("2001", IntegerType(), True),
  StructField("2002", IntegerType(), True),
  StructField("2003", IntegerType(), True),
  StructField("2004", IntegerType(), True),
  StructField("2005", IntegerType(), True),
  StructField("2006", IntegerType(), True),
  StructField("2007", IntegerType(), True),
  StructField("2008", IntegerType(), True),
  StructField("2009", IntegerType(), True),
  StructField("2010", IntegerType(), True),
  StructField("2011", IntegerType(), True),
  StructField("2012", IntegerType(), True),
  StructField("2013", IntegerType(), True),
  StructField("2014", IntegerType(), True),
  StructField("2015", IntegerType(), True),
  StructField("2016", IntegerType(), True),
  StructField("2017", IntegerType(), True)
  ])






In [8]:

Country_GDP_df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("Country_GDP.csv"), sep=",", header=True, inferSchema=True)
# Country_GDP_df.show()

print(Country_GDP_df.columns)
# Shape of the DF in spark
# print((Country_GDP_df.count(), len(Country_GDP_df.columns)))

['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code', '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968', '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017']


In [9]:
Country_GDP_df = Country_GDP_df.drop(Country_GDP_df["Indicator Name"])
Country_GDP_df = Country_GDP_df.drop(Country_GDP_df["Indicator Code"])
Country_GDP_df = Country_GDP_df.withColumnRenamed("Country Name","country_name")
Country_GDP_df = Country_GDP_df.withColumnRenamed("Country Code","country_code")

In [10]:

import numpy as np
Country_GDP_df.show()

+--------------------+------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+
|        country_name|country_code| 

In [11]:
# select the years from the past two decades.
Country_GDPdecade_df = Country_GDP_df.select("country_name","country_code","2000","2001","2002","2003","2004","2005","2006","2007","2008","2009","2010","2011",
                                          "2012","2013","2014","2015","2016","2017")
Country_GDPdecade_df.show()

# country_demo_gdp_df = Country_GDPdecade_df.toPandas()
country_demo_gdp_df = Country_GDPdecade_df.toPandas()

country_demo_gdp_df.to_csv('test.csv')


+--------------------+------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+
|        country_name|country_code|           2000|           2001|           2002|           2003|           2004|           2005|           2006|           2007|           2008|           2009|           2010|           2011|           2012|           2013|           2014|           2015|           2016|           2017|
+--------------------+------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+
|               Aruba|      

In [12]:
#Perform the aggregation of each country GDP row wise.

from pyspark.sql.functions import mean as _mean, stddev as _stddev, col

total_cols = len(Country_GDPdecade_df.columns)
total_cols = total_cols -2 
rowMean  = (sum(col(year) for year in Country_GDPdecade_df.columns[2:]) /total_cols )
rowMean
Country_GDP_mean_df = Country_GDPdecade_df.select(
    ["country_name",rowMean.alias("GDP")]
    # _stddev(col('columnName')).alias('std')
)
Country_GDP_mean_df.printSchema()
Country_GDP_mean_df.show(20)

root
 |-- country_name: string (nullable = true)
 |-- GDP: double (nullable = true)

+--------------------+--------------------+
|        country_name|                 GDP|
+--------------------+--------------------+
|               Aruba|                null|
|         Afghanistan|                null|
|              Angola|6.829992976338888...|
|             Albania|       9.856641937E9|
|             Andorra|2.9877264891666665E9|
|          Arab World|1.827444444444444...|
|United Arab Emirates|2.625555555555555...|
|           Argentina|    3.73595778014E11|
|             Armenia| 7.721111227444445E9|
|      American Samoa|                null|
| Antigua and Barbuda| 1.146689137811111E9|
|           Australia|9.728888888888889E11|
|             Austria|3.519444444444444...|
|          Azerbaijan|3.684624983427778E10|
|             Burundi|1.8658214422333333E9|
|             Belgium|4.263888888888889E11|
|               Benin| 6.378661420888889E9|
|        Burkina Faso|      7.91203

### Perform inner join between the two dataframes grouping by country_name.
### Avoid creating the duplicate columns.

In [13]:


CountryDemographics_clean_df =CountryDemographics.join(Country_GDP_mean_df,['country_name'])
CountryDemographics_clean_df.printSchema()

CountryDemographics_clean_df.show()


root
 |-- country_name: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- GDP: double (nullable = true)

+--------------------+------------+----------+-----------+--------------------+
|        country_name|country_code|  latitude|  longitude|                 GDP|
+--------------------+------------+----------+-----------+--------------------+
|             Andorra|          AD| 42.546245|   1.601554|2.9877264891666665E9|
|United Arab Emirates|          AE| 23.424076|  53.847818|2.625555555555555...|
|         Afghanistan|          AF|  33.93911|  67.709953|                null|
| Antigua and Barbuda|          AG| 17.060816| -61.796428| 1.146689137811111E9|
|             Albania|          AL| 41.153332|  20.168331|       9.856641937E9|
|             Armenia|          AM| 40.069099|  45.038189| 7.721111227444445E9|
|              Angola|          AO|-11.202692|  17.873887|6.8299929763

### Check the summary of all the dataframes so far created.

In [14]:
Country_GDP_df.summary().show()
CountryDemographics.summary().show()

CountryDemographics_clean_df.summary().show()
CountryDemographics_clean_df = CountryDemographics_clean_df.dropna()

df = CountryDemographics_clean_df.toPandas()
df.to_csv("CountryDemographics.csv")

+-------+------------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------

# Configure the RDS settings to write the cleaned dataframe into the CountryDemographics table.

In [15]:
# Configure settings for RDS

mode = "overwrite"
jdbc_url="jdbc:postgresql://womenwellbeingdatabase.cd7e8xnrkvlm.us-west-1.rds.amazonaws.com:5432/postgres"
config = {"user":"postgres", 
          "password": "postgres", 
           "driver":"org.postgresql.Driver"
         }
# Write country_clean_df to table in RDS
CountryDemographics_clean_df.printSchema()

CountryDemographics_clean_df.write.jdbc(url=jdbc_url, table='CountryGDPDemographics', mode=mode, properties=config)
CountryDemographics_clean_df.show()

root
 |-- country_name: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- GDP: double (nullable = true)

+--------------------+------------+----------+----------+--------------------+
|        country_name|country_code|  latitude| longitude|                 GDP|
+--------------------+------------+----------+----------+--------------------+
|             Andorra|          AD| 42.546245|  1.601554|2.9877264891666665E9|
|United Arab Emirates|          AE| 23.424076| 53.847818|2.625555555555555...|
| Antigua and Barbuda|          AG| 17.060816|-61.796428| 1.146689137811111E9|
|             Albania|          AL| 41.153332| 20.168331|       9.856641937E9|
|             Armenia|          AM| 40.069099| 45.038189| 7.721111227444445E9|
|              Angola|          AO|-11.202692| 17.873887|6.829992976338888...|
|           Argentina|          AR|-38.416097|-63.616672|    3.73595778014E11|
